In [22]:
import sqlite3
from sqlite3 import Error

In [23]:
def get_connection(db_file):
    """Create a database connection to a SQLite database"""
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(f"Connection to the '{db_file}' SQLite database successful!")
    except Error as e:
        print(f"Something went wrong while connecting to the '{db_file}' SQLite database!\nError: {e}")

    return conn

In [24]:
conn = get_connection("db.sqlite3")
cursor = conn.cursor()

Connection to the 'db.sqlite3' SQLite database successful!


In [25]:
# Should we apply any filter on the date as well? Will the inventory table only contain flights that are yet to take off?
# Will the inventory table also contain flights that have already taken off?

def get_affected_flights(schedule_id):
    """Returns a list of affected flights"""
    cursor.execute(
        f"""
        SELECT * 
        FROM inventory 
        WHERE schedule_id = '{schedule_id}'
        """
    )
    
    result = cursor.fetchall()
    return result

In [26]:
result = get_affected_flights('SCH-ZZ-7283355')
for row in result:
    print(f"Flight Number: {row[2]}\nAircraft Type: {row[3]}\nDeparture Date: {row[4]}\n\n")

Flight Number: 3417
Aircraft Type: Boeing 787
Departure Date: 2024-05-25


Flight Number: 3417
Aircraft Type: Boeing 787
Departure Date: 2024-05-26


Flight Number: 3417
Aircraft Type: Boeing 787
Departure Date: 2024-06-01


Flight Number: 3417
Aircraft Type: Boeing 787
Departure Date: 2024-06-02


Flight Number: 3417
Aircraft Type: Boeing 787
Departure Date: 2024-06-08


Flight Number: 3417
Aircraft Type: Boeing 787
Departure Date: 2024-06-09


Flight Number: 3417
Aircraft Type: Boeing 787
Departure Date: 2024-06-15


Flight Number: 3417
Aircraft Type: Boeing 787
Departure Date: 2024-06-16


Flight Number: 3417
Aircraft Type: Boeing 787
Departure Date: 2024-06-22


Flight Number: 3417
Aircraft Type: Boeing 787
Departure Date: 2024-06-23


Flight Number: 3417
Aircraft Type: Boeing 787
Departure Date: 2024-06-29


Flight Number: 3417
Aircraft Type: Boeing 787
Departure Date: 2024-06-30


Flight Number: 3417
Aircraft Type: Boeing 787
Departure Date: 2024-07-06


Flight Number: 3417
Aircr

In [27]:
def get_affected_passengers(schedule_id):
    cursor.execute(
        f"""
        SELECT *
        FROM schedule
        WHERE schedule_id = '{schedule_id}'
        """
    )
    
    result = cursor.fetchone()
    departure_time = result[6]
    arrival_time = result[7]
    
    affected_passengers = []
    affected_flights = get_affected_flights(schedule_id)
    for flight in affected_flights:
        cursor.execute(
            f"""
            SELECT * 
            FROM pnr 
            WHERE flight_number = '{flight[2]}'
            AND departure_date = '{flight[4]}'
            AND arrival_date = '{flight[5]}'
            AND departure_time = '{departure_time}'
            AND arrival_time = '{arrival_time}'
            """
        )
        
        result = cursor.fetchall()
        affected_passengers.extend(result)
    
    return affected_passengers

In [28]:
result = get_affected_passengers('SCH-ZZ-7283355')
for row in result:
    print(f"Passenger ID: {row[0]}")

Passenger ID: FEAPID
Passenger ID: FMGPWR
Passenger ID: FMGPWR


In [29]:
def sort_passengers(affected_passengers):
    for passenger in affected_passengers:
        cursor.execute(
            f"""
            SELECT *
            FROM passenger
            WHERE passenger_id = '{passenger[0]}'
            """
        )
        
        result = cursor.fetchall()
        for row in result:
            print(row)

    # Extract required info for each passenger from 'passenger' table
    # Calculate the priority based on the decided formula
    # Sort based on the calculated priority and return

In [30]:
result = sort_passengers(get_affected_passengers('SCH-ZZ-7283355'))


In [66]:

from datetime import datetime


def sort_flight(inv_id):
    cursor.execute(
        f"""
        SELECT *
        FROM inventory
        WHERE inventory_id = '{inv_id}'
        """
    )
    
    result = cursor.fetchone()

    schedule_id = result[1]  # Assuming schedule_id is in the second column of the result
    departure_date = result[4]  # Assuming departure_date is in the fifth column of the result
    
    cursor.execute(
        f"""
        SELECT *
        FROM inventory;
        """
    )

    result = cursor.fetchall()
    # Fetch time from schedule using schedule ID and departure date
    
    cursor.execute(
        f"""
        SELECT *
        FROM schedule
        WHERE schedule_id = '{schedule_id}'
        """
    )
    
    time_result = cursor.fetchone()
    departure_time = time_result[6]  # Assuming departure_time is in the first column of the time_result
    src = time_result[4]
    dest = time_result[5]


    inv_dict = {}  # Create an empty dictionary to store inv_ids and priorities
    
    for row in result:
        inv = row[0]

        if(inv == inv_id):
            continue

        # Fetch time from schedule using schedule ID and departure date
        cursor.execute(
            f"""
            SELECT *
            FROM schedule
            WHERE schedule_id = '{row[1]}'
            """
        )
        res = cursor.fetchone()
        time = res[6]
        src_ap = res[4]
        dest_ap = res[5]
        
        date = row[4]

        if date < departure_date or (date == departure_date and time < departure_time) or src != src_ap or dest != dest_ap:
            continue
        
        # Calculate priority
        priority = 0

        departure_datetime = datetime.strptime(departure_date + ' ' + departure_time, '%Y-%m-%d %H:%M')
        current_datetime = datetime.strptime(date + ' ' + time, '%Y-%m-%d %H:%M')

        # Calculate the time difference in hours
        time_difference = (current_datetime - departure_datetime).total_seconds() / 3600
        if time_difference <= 6:
            priority = 70
        elif time_difference <= 12:
            priority = 50
        elif time_difference <= 24:
            priority = 40
        elif time_difference <= 48:
            priority = 30

        inv_dict[inv] = priority
    
    return inv_dict

result = sort_flight('INV-ZZ-1875559')
print(result)


{'INV-ZZ-2847882': 40, 'INV-ZZ-2048509': 0, 'INV-ZZ-9629978': 0, 'INV-ZZ-6952251': 0, 'INV-ZZ-9701617': 0, 'INV-ZZ-4703448': 0, 'INV-ZZ-9289613': 0, 'INV-ZZ-3502758': 0, 'INV-ZZ-8851705': 0, 'INV-ZZ-5391474': 0, 'INV-ZZ-8820852': 0, 'INV-ZZ-7070488': 0, 'INV-ZZ-7396098': 0, 'INV-ZZ-2962005': 0, 'INV-ZZ-2735784': 0, 'INV-ZZ-1671415': 0, 'INV-ZZ-3652962': 0, 'INV-ZZ-2937163': 0, 'INV-ZZ-1350452': 0, 'INV-ZZ-3541111': 0, 'INV-ZZ-4570768': 0, 'INV-ZZ-9069594': 0, 'INV-ZZ-4319174': 0, 'INV-ZZ-8429501': 0, 'INV-ZZ-7155087': 0, 'INV-ZZ-4243562': 0}
